# DATA20001 Deep Learning - Group Project
## Image project

- Testing [resnext101_32x8d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning models
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied
- No data balancing applied



In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

## Import dependencies and select correct device

In [2]:
# import dependencies
import train_model
import eval_model
import utils
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation

import time
import torch
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [3]:
device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using GPU


## Define datasets

- shuffle
- train: 17k
- valid: 1.5k
- test: 1.5k

In [4]:
mapping_file = "file_to_labels_table.csv"
df = pd.read_csv(mapping_file)
train_idx, val_idx = utils.get_train_val_indexes(df, 0.85, shuffle=True)
val_test_split_idx = int(val_idx.shape[0]*.5)
test_idx = val_idx[:val_test_split_idx]
val_idx = val_idx[val_test_split_idx:]

dataAugmentation = DataAugmentation()
trainDataset = ImageDataset(train_idx, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(val_idx)
testDataset = ImageDataset(test_idx)

print("Saving test dataset temporarily so test can be performed after restarting kernel.")
utils.save_pickle(testDataset, "tmp_test_data_exp_transfer_learning_5.pkl")
print("# Train:", len(trainDataset))
print("# Valid:", len(valDataset))
print("# Test:", len(testDataset))

Saving test dataset temporarily so test can be performed after restarting kernel.
# Train: 17000
# Valid: 1500
# Test: 1500


## Define the model architecture

The initial transfer learning implementation

In [5]:
torch.cuda.empty_cache()

preTrainedModel = models.resnext101_32x8d(pretrained=True)

for param in preTrainedModel.parameters():
    param.requires_grad = False

num_ftrs = preTrainedModel.fc.in_features
num_classes = 14

preTrainedModel.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, num_classes),
    torch.nn.Sigmoid()
)

model = preTrainedModel.to(device)

## Define test case

Hyperparam search could be defined here. No hyperparam searches in this notebook as purpose of this test is to test the effect of data preprocessing technique on initial transfer learning model architecture and see how longer run on epochs affect loss.

**Test objective**:

- Testing [resnext101_32x8d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning model
- Data preprocessing includes 
  - data augmentation, 
  - shuffle 
  - split train=17k, val=1.5k, test=1.5k


**Training results**:

- Best validation results:
 - Loss: 0.1113 Accuracy: 0.9619
 - Precision: 0.7223 Recall: 0.7258
 - F1: 0.7241 Duration: 2m 3s
- Testing set results.
 - Accuracy 0.963
 - Precision 0.745
 - Recall 0.742
 - F1-score 0.743

In [6]:
n_epochs = 6

model, logs = train_model.train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs=n_epochs,
    returnLogs=True
)

Epoch 0/6
----------
Validating...

Progress: 100%

Loss: 0.7369 Accuracy: 0.4267
Precision: 0.0667 Recall: 0.5628
F1: 0.1192 Duration: 1m 15s

Epoch 1/6
----------
Training...

Progress: 100%

Loss: 0.1413 Accuracy: 0.9498
Precision: 0.7214 Recall: 0.5006
F1: 0.5911 Duration: 22m 9s

Validating...

Progress: 100%

Loss: 0.1146 Accuracy: 0.9572
Precision: 0.6798 Recall: 0.7169
F1: 0.6978 Duration: 1m 33s

Epoch 2/6
----------
Training...

Progress: 100%

Loss: 0.1315 Accuracy: 0.9533
Precision: 0.7288 Recall: 0.5665
F1: 0.6375 Duration: 19m 53s

Validating...

Progress: 100%

Loss: 0.1205 Accuracy: 0.9580
Precision: 0.7339 Recall: 0.6133
F1: 0.6682 Duration: 1m 40s

Epoch 3/6
----------
Training...

Progress: 100%

Loss: 0.1281 Accuracy: 0.9548
Precision: 0.7379 Recall: 0.5850
F1: 0.6526 Duration: 17m 43s

Validating...

Progress: 100%

Loss: 0.1057 Accuracy: 0.9620
Precision: 0.7295 Recall: 0.7134
F1: 0.7214 Duration: 1m 27s

Epoch 4/6
----------
Training...

Progress: 100%

Loss: 0.1

## Save model and logs

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [7]:
pd.DataFrame(logs["val"]).to_csv("exp_transfer_learning_5_logs_val.csv", index=False)
pd.DataFrame(logs["train"]).to_csv("exp_transfer_learning_5_logs_train.csv", index=False)
torch.save(model, 'exp_transfer_learning_5_model.pkl')  # saves model with architecture

## Test model

- Load model back from state_dict and perform test on testing set
- Load test-set
- Load training time logs of performance

In [8]:
model = torch.load("exp_transfer_learning_5_model.pkl")
testDataset = utils.load_pickle("tmp_test_data_exp_transfer_learning_5.pkl")
logsTable = pd.read_csv("exp_transfer_learning_5_logs_val.csv")
y_hats, y_trues = eval_model.test_model(model, testDataset, device)

### Validation set performance during training

In [9]:
logsTable

,duration,loss,accuracy,precision,recall,f1,true positives,true negative,false positives,false negatives
0,75.167367,0.736887,0.426667,0.066683,0.562845,0.119239,815,8145,11407,633
1,92.598319,0.114587,0.957190,0.679764,0.716851,0.697815,1038,19063,489,410
2,100.321687,0.120497,0.958000,0.733884,0.613260,0.668172,888,19230,322,560
3,87.132280,0.105652,0.962000,0.729520,0.713398,0.721369,1033,19169,383,415
4,78.625497,0.120770,0.959714,0.698026,0.732735,0.714960,1061,19093,459,387
5,122.784722,0.111285,0.961857,0.722337,0.725829,0.724079,1051,19148,404,397
6,83.861381,0.118159,0.960238,0.707515,0.721685,0.714530,1045,19120,432,403


### Testing set performance

In [10]:
print("Testing set results.")
print(f" Accuracy\t{round(eval_model.get_metric(y_trues, y_hats, 'accuracy'), 3)}")
print(f" Precision\t{round(eval_model.get_metric(y_trues, y_hats, 'precision'), 3)}")
print(f" Recall \t{round(eval_model.get_metric(y_trues, y_hats, 'recall'), 3)}")
print(f" F1-score\t{round(eval_model.get_metric(y_trues, y_hats, 'f1'), 3)}")

Testing set results.
 Accuracy	0.963
 Precision	0.745
 Recall 	0.742
 F1-score	0.743


### Extra analysis

How many positives and negatives has been predicted.

Tuples like: **(label, count)**

In [11]:
list(zip(*np.unique(y_hats, return_counts=True)))

[(0, 19493), (1, 1507)]

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [12]:
np.savetxt('results.txt', y, fmt='%d')

NameError: name 'y' is not defined